In [ ]:
import cv2
import numpy as np
import heapq
import requests
import json
import re
from datetime import datetime
from textwrap import wrap

# --------------------------------------------------------------
# FLOOD EVACUATION PLANNER + MAS AGENT INFLUENCE VISUALIZER
# Click A -> Click B -> MAS runs -> shows path + per-agent influence & reasoning
# --------------------------------------------------------------

# --------------------- CONFIG ---------------------
GRID_NPY = "safe_grid_4.npy"
ORIGINAL_IMAGE = "flood_4.jpg"   # optional: if mismatch size it will fallback to colored
OUTPUT_IMAGE = "flood_path_ai_result_4.png"

# Colors (BGR)
COLOR_FLOOD = (255, 0, 0)      # blue water
COLOR_SAFE  = (42, 42, 165)    # land
COLOR_PATH  = (0, 255, 0)      # green path
COLOR_A     = (0, 255, 255)    # yellow A marker
COLOR_B     = (203, 192, 255)  # pink B marker

# influence overlay colors (BGR)
COLOR_PENALIZE = (0, 0, 200)   # red-ish overlay (penalized cells)
COLOR_REWARD   = (0, 200, 0)   # green overlay (rewarded cells)
COLOR_AVOID    = (0, 200, 200) # yellow/cyan overlay (avoid cells)

# visualization parameters
MAX_DRAW_CELLS = 3000   # avoid drawing enormous overlays
OVERLAY_ALPHA = 0.45

# --------------------- LOAD GRID & VIS ---------------------
grid = np.load(GRID_NPY)
H, W = grid.shape

# RECONSTRUCT VISUAL EXACTLY LIKE YOUR FIRST IMAGE
vis_base = np.zeros((H, W, 3), dtype=np.uint8)
vis_base[grid == 0] = (0, 0, 150)        # Dark red = flooded (matches your reference)
vis_base[grid == 1] = (50, 100, 50)      # Greenish-brown = safe land

# Blend with real satellite image for perfect visual match
bg_img = cv2.imread(ORIGINAL_IMAGE)
if bg_img is not None and bg_img.shape[:2] == (H, W):
    vis_base = cv2.addWeighted(bg_img, 0.6, vis_base, 0.4, 0)

vis = vis_base.copy()

# Update colors used in legend and path
COLOR_FLOOD = (0, 0, 150)
COLOR_SAFE  = (50, 100, 50)

# --------------------- OLLAMA COMMUNICATION ---------------------
def call_ollama(model, prompt, temperature=0.3):
    """
    Simple Ollama call returning raw text response or None on failure.
    Keep it the same as your previously used call.
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": temperature, "num_ctx": 4096}
    }
    try:
        r = requests.post(url, json=payload, timeout=120)
        r.raise_for_status()
        return r.json().get("response", "").strip()
    except Exception as e:
        print(f"Ollama error ({model}): {e}")
        return None

# --------------------- HELPERS ---------------------
coord_re = re.compile(r'\(?\s*(-?\d+)\s*,\s*(-?\d+)\s*\)?')

def parse_coords_from_text(text, max_coords=5000):
    """
    Extract (x,y) integer coords from free text robustly using regex.
    Returns list of (x,y) tuples. Respects boundaries will be checked by caller.
    """
    if not text:
        return []
    coords = []
    for m in coord_re.findall(text):
        try:
            x = int(m[0]); y = int(m[1])
            coords.append((x, y))
            if len(coords) >= max_coords:
                break
        except:
            continue
    return coords

def safe_coord_list(coords):
    """Filter coords to valid in-grid coordinates"""
    out = []
    for x, y in coords:
        if 0 <= x < W and 0 <= y < H:
            out.append((x, y))
    return out

def draw_overlay(base_img, coords, color, alpha=0.45, max_draw=MAX_DRAW_CELLS):
    """Draw semi-transparent overlay for a set of coords onto base_img (returns new img)"""
    overlay = base_img.copy()
    draw_count = 0
    for x, y in coords:
        if draw_count >= max_draw:
            break
        # draw small filled rectangle for visibility (1x1 pixel scaled)
        overlay[y, x] = color
        draw_count += 1
    combined = cv2.addWeighted(overlay, alpha, base_img, 1 - alpha, 0)
    return combined

def small_text_window(title, lines, size=(700, 800)):
    """Create an OpenCV image with wrapped text lines and display in a named window."""
    w, h = size
    img = np.ones((h, w, 3), dtype=np.uint8) * 18
    margin = 12
    y = 24
    cv2.putText(img, title, (margin, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (230,230,230), 2)
    y += 28
    max_lines = (h - y) // 16
    wrapped_lines = []
    for line in lines:
        wrapped_lines.extend(wrap(line, 90))
    for i, line in enumerate(wrapped_lines[-max_lines:]):
        cv2.putText(img, line, (margin, y + i*16), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (210,210,210), 1)
    cv2.imshow(title, img)

# --------------------- MULTI-AGENT COST GRID REFINEMENT (with influence recording) ---------------------
def mas_refine_cost_grid(start_px, goal_px):
    """
    Runs 4 agents. Returns:
      - cost_grid: numpy array with modified costs
      - agent_infos: list of dicts with keys:
           'name', 'raw_output' (string), 'coords' (parsed list), 'applied_count', 'action' ('penalize'/'reward'/'avoid'), 'effect'
      - commander_text: string output of agent 4 (final decision)
    """
    tstamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"\n[M A S] RUN @ {tstamp} -- start {start_px} goal {goal_px}")

    # base cost: safe cells = 1.0, flood = very large (impassable)
    cost_grid = np.ones((H, W), dtype=np.float32) * 999.0
    cost_grid[grid == 1] = 1.0
    cost_grid[grid == 0] = 99999.0

    agent_infos = []

    # Describe surrounding area for cmd prompts (short)
    def describe_zone(x, y, r=40):
        y0, y1 = max(0, y-r), min(H, y+r)
        x0, x1 = max(0, x-r), min(W, x+r)
        zone = grid[y0:y1, x0:x1]
        safe = int(np.sum(zone == 1))
        total = zone.size
        flood = total - safe
        return {"safe_pct": safe/total if total else 0.0, "safe_cells": safe, "flood_cells": flood, "area": total}

    start_desc = describe_zone(*start_px)
    goal_desc  = describe_zone(*goal_px)

    # ---------------- AGENT 1: Flood Dynamics (penalize predicted cells + reason) ----------------
    print("  Agent 1: Flood Dynamics -> predicting imminent risk")
    prompt1 = f"""
You are a hydrologist. Predict which currently SAFE cells will flood in the next 30–60 minutes.
Start zone: {json.dumps(start_desc)}
Goal zone:  {json.dumps(goal_desc)}
Return a plain Python-style list of (x,y) pairs to penalize heavily (+80 cost).
If none, return [].
Example: [(512,340), (520,355), ...]
Also include one short sentence explanation at the end.
"""
    out1 = call_ollama("gpt-oss:120b-cloud", prompt1) or "[]"
    coords1 = parse_coords_from_text(out1, max_coords=5000)
    coords1 = safe_coord_list(coords1)
    applied1 = 0
    for x, y in coords1[:150]:
        if grid[y, x] == 1:
            cost_grid[y, x] += 80.0
            applied1 += 1
    reason1 = out1.strip()
    agent_infos.append({
        "name": "Agent 1",
        "role": "Flood Dynamics",
        "raw_output": reason1,
        "coords": coords1,
        "applied_count": applied1,
        "action": "penalize",
        "effect": "+80 cost (on safe cells)"
    })

    # ---------------- AGENT 2: Corridor Strategist (reward wide corridors) ----------------
    print("  Agent 2: Corridor Strategist -> rewarding wide corridors")
    prompt2 = f"""
You are an evacuation route planner.
Given start zone: {json.dumps(start_desc)} and goal zone: {json.dumps(goal_desc)},
return a Python-style list of up to 300 (x,y) coordinates on SAFE land that you want to reward (lower cost by 0.5).
Also include one short sentence explaining why you picked them.
"""
    out2 = call_ollama("gpt-oss:120b-cloud", prompt2) or "[]"
    coords2 = parse_coords_from_text(out2, max_coords=5000)
    coords2 = safe_coord_list(coords2)
    applied2 = 0
    for x, y in coords2[:300]:
        if grid[y, x] == 1:
            cost_grid[y, x] = max(0.3, cost_grid[y, x] - 0.5)
            applied2 += 1
    reason2 = out2.strip()
    agent_infos.append({
        "name": "Agent 2",
        "role": "Corridor Strategist",
        "raw_output": reason2,
        "coords": coords2,
        "applied_count": applied2,
        "action": "reward",
        "effect": "-0.5 cost (min 0.3)"
    })

    # ---------------- AGENT 3: Human Behavior Expert (penalize confusing/choke points) ----------------
    print("  Agent 3: Human Behavior -> penalizing panic zones / choke points")
    prompt3 = f"""
You are a disaster psychologist.
Given start zone: {json.dumps(start_desc)} and goal zone: {json.dumps(goal_desc)},
return a Python-style list of (x,y) coordinates (up to 120) that are confusing, narrow or panic-prone which should get a penalty of +25 cost.
Also include a one-line explanation.
"""
    out3 = call_ollama("gpt-oss:120b-cloud", prompt3) or "[]"
    coords3 = parse_coords_from_text(out3, max_coords=5000)
    coords3 = safe_coord_list(coords3)
    applied3 = 0
    for x, y in coords3[:120]:
        if grid[y, x] == 1:
            cost_grid[y, x] += 25.0
            applied3 += 1
    reason3 = out3.strip()
    agent_infos.append({
        "name": "Agent 3",
        "role": "Human Behavior Expert",
        "raw_output": reason3,
        "coords": coords3,
        "applied_count": applied3,
        "action": "avoid",
        "effect": "+25 cost"
    })

    # ---------------- AGENT 4: Incident Commander (final decision/explanation) ----------------
    print("  Agent 4: Incident Commander -> final directive")
    prompt4 = f"""
You are the Incident Commander. Summarize in one short sentence the chosen approach given the other agents' suggested modifications.
Return only a single sentence explaining which corridor/approach should be used and why.
"""
    out4 = call_ollama("gpt-oss:120b-cloud", prompt4) or "No commander response."
    commander_text = out4.strip()
    agent_infos.append({
        "name": "Agent 4",
        "role": "Incident Commander",
        "raw_output": commander_text,
        "coords": [],
        "applied_count": 0,
        "action": "command",
        "effect": commander_text
    })

    # Summarize counts and return
    return cost_grid, agent_infos

# --------------------- A* USING AI COST GRID ---------------------
def heuristic(a, b):
    # Manhattan heuristic
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar_mas(start, goal, cost_grid):
    neighbors = [(0,1),(1,0),(0,-1),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]
    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal), 0, start))
    came_from = {}
    g_score = {start: 0}
    seen = set()

    while open_set:
        _, _, current = heapq.heappop(open_set)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        if current in seen:
            continue
        seen.add(current)

        for dx, dy in neighbors:
            nx, ny = current[0] + dx, current[1] + dy
            if not (0 <= nx < W and 0 <= ny < H):
                continue
            # only traverse safe cells (grid==1)
            if grid[ny, nx] != 1:
                continue
            if cost_grid[ny, nx] >= 9999:
                continue
            step_cost = cost_grid[ny, nx] * (1.414 if dx and dy else 1.0)
            tent_g = g_score[current] + step_cost
            neighbor = (nx, ny)
            if neighbor not in g_score or tent_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tent_g
                f = tent_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f, tent_g, neighbor))
    return None

# --------------------- DETAILED AGENT REASONING EXPANSION ---------------------
def expand_agent_reasoning(ag, start, goal, path, cost_grid, max_example_coords=6):
    """
    Given an agent info dict, produce at least 4 lines (strings) describing:
      - short summary of role & high-level action
      - what it actually changed/applied (counts, effect)
      - show a few example coordinates it affected (if any)
      - an interpretation of how this likely influenced the final path
    Returns list of lines (>=4).
    """
    name = ag.get("name", "Agent")
    role = ag.get("role", "")
    raw = ag.get("raw_output", "").strip()
    action = ag.get("action", "n/a")
    applied = ag.get("applied_count", 0)
    effect = ag.get("effect", "")

    lines = []

    # 1) header summary
    header = f"{name} ({role}) — action: {action}  effect: {effect}"
    lines.append(header)

    # 2) what it changed
    lines.append(f"It applied its logic to {applied} cell(s) (reported).")

    # 3) short raw output preview (1-2 lines)
    if raw:
        raw_preview = raw.splitlines()
        preview_line = raw_preview[0][:220] if raw_preview else ""
        lines.append("Agent raw summary: " + (preview_line if preview_line else "[no details]"))
        # include second raw line if available
        if len(raw_preview) > 1:
            lines.append("  " + raw_preview[1][:220])
    else:
        lines.append("Agent raw summary: [no raw text produced]")

    # 4) example coordinates affected (if present)
    coords = ag.get("coords", []) or []
    if coords:
        sample_coords = coords[:max_example_coords]
        coords_text = ", ".join([f"({x},{y})" for x, y in sample_coords])
        lines.append(f"Example affected cells (truncated): {coords_text}")
    else:
        lines.append("No coordinate list produced by agent (or list empty).")

    # 5) inferred impact on path (best-effort)
    impact = ""
    if path and coords:
        coords_set = set(coords)
        hit = sum(1 for p in path if p in coords_set)
        impact = f"This agent's cells intersected {hit} path cell(s)." 
    elif path and not coords:
        impact = "Path present but agent provided no coordinates to compute intersections."
    else:
        impact = "No final path to evaluate intersections."
    lines.append(impact)

    # Ensure at least 4 lines (we typically have 5-6)
    if len(lines) < 4:
        # pad with inference lines
        lines.append("Additional inference: agent adjusted local costs to bias route selection.")
        lines.append("Confidence: heuristic inferred from applied cost modifications.")

    # Trim/limit to reasonable size; small_text_window will wrap.
    return lines

# --------------------- VISUAL WINDOWS for agent outputs & influence ---------------------
def show_mas_reasoning_window(agent_infos, start, goal, path, cost_grid):
    """
    Show a textual reasoning window summarizing each agent's raw output,
    as well as a short narrative about how they influenced the final path in,.
    Each agent is expanded into at least 4 descriptive lines.
    This should be given in such a way that people who read the reasoning window shoul understand what it means.
    """
    tstamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lines = [f"MAS RUN: {tstamp}", f"Start: {start}   Goal: {goal}", ""]

    # For each agent, expand into multi-line block
    for ag in agent_infos:
        expanded = expand_agent_reasoning(ag, start, goal, path, cost_grid)
        # Add a separator header and the expanded block
        lines.append("=" * 60)
        lines.append(f"{ag.get('name','Agent')}:")
        lines.extend(expanded)
        lines.append("")  # blank line between agents

    # Overall path summary
    lines.append("=" * 60)
    if path:
        lines.append(f"Final path steps: {len(path)}")
        # For each agent, report intersections (if coords available)
        for ag in agent_infos:
            if ag.get("coords"):
                coords_set = set(ag["coords"])
                hit = sum(1 for p in path if p in coords_set)
                lines.append(f"  {ag.get('name')}: intersects {hit} path cell(s).")
    else:
        lines.append("No path found; MAS prevented any safe route.")

    # Show window (size chosen to allow longer output)
    small_text_window("MAS Reasoning (Agent Influence & Detailed Output)", lines, size=(880, 980))


def show_agent_influence_overlay(agent_infos, path):
    """
    Create a visualization showing:
      - base map
      - penalized cells overlay (red)
      - rewarded cells overlay (green)
      - avoid cells overlay (yellow)
      - final path (green dots)
      - A/B markers
    """
    overlay = vis_base.copy()

    # gather up to MAX_DRAW_CELLS from each agent type
    penalize_coords = []
    reward_coords   = []
    avoid_coords    = []
    for ag in agent_infos:
        if ag["action"] == "penalize":
            penalize_coords.extend(ag["coords"])
        elif ag["action"] == "reward":
            reward_coords.extend(ag["coords"])
        elif ag["action"] == "avoid":
            avoid_coords.extend(ag["coords"])

    # filter to safe cells and unique
    penalize_coords = list({(x,y) for x,y in safe_coord_list(penalize_coords)})[:MAX_DRAW_CELLS]
    reward_coords   = list({(x,y) for x,y in safe_coord_list(reward_coords)})[:MAX_DRAW_CELLS]
    avoid_coords    = list({(x,y) for x,y in safe_coord_list(avoid_coords)})[:MAX_DRAW_CELLS]

    # draw overlays one by one (composed)
    img = overlay.copy()
    if penalize_coords:
        img = draw_overlay(img, penalize_coords, COLOR_PENALIZE, alpha=OVERLAY_ALPHA)
    if reward_coords:
        img = draw_overlay(img, reward_coords, COLOR_REWARD, alpha=OVERLAY_ALPHA)
    if avoid_coords:
        img = draw_overlay(img, avoid_coords, COLOR_AVOID, alpha=OVERLAY_ALPHA)

    # draw path
    if path:
        for (px, py) in path:
            cv2.circle(img, (px, py), 2, COLOR_PATH, -1)

    # draw A/B markers if present
    # We will rely on the last A/B drawn in global vis (but ensure visible)
    # Just show a small legend text overlay:
    h, w = img.shape[:2]
    panel_w = 360
    overlayp = img.copy()
    cv2.rectangle(overlayp, (w - panel_w - 12, 12), (w - 12, 200), (8,8,8), -1)
    cv2.addWeighted(overlayp, 0.6, img, 0.4, 0, img)
    ox = w - panel_w
    oy = 36
    cv2.putText(img, "Agent Influence Overlay", (ox + 6, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (230,230,230), 2)
    oy += 28
    cv2.putText(img, "Penalized (Agent1)  -> red overlay", (ox + 8, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.45, COLOR_PENALIZE, 1)
    oy += 20
    cv2.putText(img, "Rewarded (Agent2)   -> green overlay", (ox + 8, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.45, COLOR_REWARD, 1)
    oy += 20
    cv2.putText(img, "Avoid (Agent3)      -> yellow overlay", (ox + 8, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.45, COLOR_AVOID, 1)
    oy += 22
    cv2.putText(img, "Path (final)        -> small green dots", (ox + 8, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.45, COLOR_PATH, 1)

    cv2.imshow("Agent Influence Overlay", img)

# --------------------- MOUSE CALLBACK ---------------------
points = []
window_name = "AI Flood Evacuation Planner – MAS Pathfinding"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, min(1200, W), min(800, H))

def draw_legend_on_image(img):
    """small legend on the image (top-right)"""
    x = W - 370
    y = 15
    overlay = img.copy()
    cv2.rectangle(overlay, (x - 10, y), (W - 10, y + 145), (10, 10, 10), -1)
    cv2.addWeighted(overlay, 0.75, img, 0.25, 0, img)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, "LEGEND", (x, y + 28), font, 0.8, (255, 255, 255), 2)
    items = [
        (y + 60,  "Flooded (Unsafe)", COLOR_FLOOD),
        (y + 100, "Safe Land",       COLOR_SAFE),
        (y + 140, "AI Safe Path",    COLOR_PATH)
    ]
    for py, text, color in items:
        cv2.rectangle(img, (x, py - 20), (x + 28, py), color, -1)
        cv2.putText(img, text, (x + 38, py - 2), font, 0.7, (255, 255, 255), 2)

def click_event(event, x, y, flags, param):
    global points, vis
    if event == cv2.EVENT_LBUTTONDOWN:
        if not (0 <= x < W and 0 <= y < H):
            return
        points.append((x, y))

        # Mark A or B
        if len(points) == 1:
            # reset vis to base to avoid stacking many markers
            vis = vis_base.copy()
            cv2.circle(vis, (x, y), 14, COLOR_A, -1)
            cv2.putText(vis, "A", (x + 18, y + 28), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,0), 4)
            print(f"[UI] Start A set: {x,y}")
            tmp = vis.copy()
            draw_legend_on_image(tmp)
            cv2.imshow(window_name, tmp)

        elif len(points) == 2:
            ax, ay = points[0]
            bx, by = points[1]
            # reset vis to base and draw both A & B after path computed
            vis = vis_base.copy()
            cv2.circle(vis, (ax, ay), 14, COLOR_A, -1)
            cv2.putText(vis, "A", (ax + 18, ay + 28), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,0), 4)
            cv2.circle(vis, (bx, by), 14, COLOR_B, -1)
            cv2.putText(vis, "B", (bx + 18, by + 28), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,0), 4)
            print(f"[UI] Goal B set: {bx,by}")

            # Ensure both are on safe cells
            if grid[ay, ax] != 1:
                print("[MAS] ERROR: Start A is not on SAFE land. Pick A on safe land.")
                lines = ["ERROR: Start A is not SAFE land. Please pick A on safe land."]
                small_text_window("MAS Reasoning (ERROR)", lines, size=(600,150))
                points.clear()
                cv2.imshow(window_name, vis)
                return
            if grid[by, bx] != 1:
                print("[MAS] ERROR: Goal B is not on SAFE land. Pick B on safe land.")
                lines = ["ERROR: Goal B is not SAFE land. Please pick B on safe land."]
                small_text_window("MAS Reasoning (ERROR)", lines, size=(600,150))
                points.clear()
                cv2.imshow(window_name, vis)
                return

            # Run MAS -> get modified cost grid and agent infos
            cost_grid, agent_infos = mas_refine_cost_grid((ax, ay), (bx, by))

            # Run A* on cost_grid (only safe cells allowed)
            path = astar_mas((ax, ay), (bx, by), cost_grid)

            # Draw path onto vis copy
            vis_result = vis.copy()
            if path:
                for (px, py) in path:
                    cv2.circle(vis_result, (px, py), 4, COLOR_PATH, -1)
                print(f"[MAS] Path found: {len(path)} steps")
            else:
                cv2.putText(vis_result, "NO SURVIVABLE ROUTE , YOUR'E DEAD", (W//4, H//2),
                           cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,0,255), 6)
                print("[MAS] No survivable route found.")

            # Show the influence overlay window (visual)
            show_agent_influence_overlay(agent_infos, path)

            # Show textual reasoning window describing each agent's raw output and how many path cells were intersected
            show_mas_reasoning_window(agent_infos, (ax, ay), (bx, by), path, cost_grid)

            # Update main window (legend + final)
            tmp = vis_result.copy()
            draw_legend_on_image(tmp)
            cv2.imshow(window_name, tmp)

            # Clear points so user can pick new A/B
            points.clear()

# --------------------- MAIN ---------------------
cv2.setMouseCallback(window_name, click_event)
# initial show
tmp = vis_base.copy()
draw_legend_on_image(tmp)
cv2.imshow(window_name, tmp)
print("Click once to set START (A). Click again to set GOAL (B).")
print("After MAS runs, three windows will appear:")
print(" - main map with final path")
print(" - 'Agent Influence Overlay' (visual overlay of penalized/rewarded/avoid cells)")
print(" - 'MAS Reasoning (Agent Influence & Raw Output)' (textual explanation)")

# event loop
while True:
    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC
        break

cv2.destroyAllWindows()


Click once to set START (A). Click again to set GOAL (B).
After MAS runs, three windows will appear:
 - main map with final path
 - 'Agent Influence Overlay' (visual overlay of penalized/rewarded/avoid cells)
 - 'MAS Reasoning (Agent Influence & Raw Output)' (textual explanation)
[UI] Start A set: (200, 107)
[UI] Goal B set: (448, 643)

[M A S] RUN @ 2025-12-01 06:22:51 -- start (200, 107) goal (448, 643)
  Agent 1: Flood Dynamics -> predicting imminent risk
  Agent 2: Corridor Strategist -> rewarding wide corridors
  Agent 3: Human Behavior -> penalizing panic zones / choke points
  Agent 4: Incident Commander -> final directive
[MAS] Path found: 1452 steps
